In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib as plt
import scipy as sp
%matplotlib inline
from scipy import stats
import datetime
from datetime import timedelta

In [ ]:
responses1 = pd.read_csv('data/Responses/responsesP1(Jan-Feb).csv')
responses2 = pd.read_csv('data/Responses/responsesP2(Mar-Apr).csv')
responses3 = pd.read_csv('data/Responses/responsesP3(May-Jun).csv')
responses4 = pd.read_csv('data/Responses/responsesP4(Jul-Aug).csv')
responses5 = pd.read_csv('data/Responses/responsesP5(Sep-Oct).csv')
responses6 = pd.read_csv('data/Responses/responsesP6(Nov-Dec).csv')

responses1.head()

In [ ]:
incidents1 = pd.read_csv('data/Incidents/incidentsQ1.csv')
incidents2 = pd.read_csv('data/Incidents/incidentsQ2.csv')
incidents3 = pd.read_csv('data/Incidents/incidentsQ3.csv')
incidents4 = pd.read_csv('data/Incidents/incidentsQ4.csv')

In [ ]:
incidents = pd.DataFrame()
incidents = incidents.append(incidents1).append(incidents2).append(incidents3).append(incidents4)

In [ ]:
responses = pd.DataFrame()
responses = responses.append(responses1).append(responses2).append(responses3).append(responses4).append(responses5).append(responses6)

In [ ]:
responses.shape

In [ ]:
# If "arrived at scene" is null, that means the initial dispatch assignment 
# was cancelled and another vehicle was assigned - should drop (~570k)
responses.dropna(subset = ['arrivedatscene'],inplace=True)
responses.shape

In [ ]:
#sanity check - this combination should return no results
responses[responses['arrivedathospital'].isnull() & responses['tohospital'].notnull()]

In [ ]:
responses[responses['callsignkey']==22407].sort_values(['dispatch'])

In [ ]:
vehicleIDs = responses.callsignkey.unique()
df = pd.DataFrame()
df = df.append(responses[responses['callsignkey']==22344].sort_values(['dispatch']))

In [ ]:
#calculate idle time between greentime and next dispatch
df = responses[responses['callsignkey']==19621].sort_values(['dispatch'])
df[["callstart", "dispatch", "arrivedatscene", "leftscene","arrivedathospital","greentime"]
  ] = df[["callstart", "dispatch", "arrivedatscene","leftscene",
                        "arrivedathospital","greentime"]].apply(pd.to_datetime)
df['idleTime'] = df['dispatch'] - df['greentime'].shift(1)
df.head(50)

In [ ]:
df.shape

In [ ]:
#calculate idle time between greentime and next dispatch

vehicleIDs = responses.callsignkey.unique()
df = pd.DataFrame()

for key in vehicleIDs:
    #print(key)
    df_temp = responses[responses['callsignkey']==key].sort_values(['dispatch'])
    df_temp[["callstart", "dispatch", "arrivedatscene", "leftscene","arrivedathospital",
             "greentime"]] = df_temp[["callstart", "dispatch", "arrivedatscene","leftscene",
                                 "arrivedathospital","greentime"]].apply(pd.to_datetime)
    df_temp['idleTime'] = df_temp['dispatch'] - df_temp['greentime'].shift(1)
    df = df.append(df_temp)
    
df.shape

In [ ]:
df.to_csv('responsesWithIdleTime.csv')

In [ ]:
df.head()

In [ ]:
#filter shift changes
df_regHours = df[df['callstart'].dt.hour>5]

a = datetime.timedelta(hours=10)
df_regHours = df[df['idleTime']<a]

In [ ]:
df_regHours['idleTime'].median()

In [ ]:
(df_regHours['idleTime'] / pd.Timedelta(minutes=1)).plot.hist(bins=1000, xlim=(-5,10))

In [ ]:
responses[responses['callsignkey']==22344]

In [ ]:
incidents['incidentid'].nunique()

In [ ]:
responses['incidentid'].nunique()

In [ ]:
ccgCount = responses.groupby('ccgcluster')
incidentCountsCCG = ccgCount.count()['incidentid'].sort_values(ascending=False)
incidentCountsCCG.to_csv('incidentCountsCCG.csv')

boroughCount = responses.groupby('borough')
incidentCountsBorough = boroughCount.count()['incidentid'].sort_values(ascending=False)
incidentCountsBorough.to_csv('incidentCountsBorough.csv')

wardCount = responses.groupby('wardID','wardname')
incidentCountsWard = wardCount.count()['incidentid'].sort_values(ascending=False)
incidentCountsWard.to_csv('incidentCountsWard.csv')

lsoaCount = responses.groupby('lsoa')
incidentCountsLSOA = lsoaCount.count()['incidentid'].sort_values(ascending=False)
incidentCountsLSOA.to_csv('incidentCountsLSOA.csv')

In [ ]:
incidentCountsCCG

In [ ]:
incidentCountsBorough

In [ ]:
incidentCountsLSOA = lsoaCount.count()['incidentid'].sort_values(ascending=False)

In [ ]:
lsoaCount.mean()['response']

In [ ]:
wardCount = responses.groupby(['wardID','wardname'])
incidentCountsWard = wardCount.count()['incidentid']


In [ ]:
incidentCountsWard.sort_values(ascending=False)

In [ ]:
wardCount = responses.groupby(['wardID','wardname'])
incidentCountsWard = wardCount.count()['incidentid'].sort_values(ascending=False)
incidentCountsWard.to_csv('incidentCountsWard.csv')

In [ ]:
incidentCountsWard.to_csv('incidentCountsWard.csv')

## Proportion of activation/mobilisation/running time of total response time

In [ ]:
listVars = ['response','activation', 'mobilisation','running']
responses[responses['dohcategory'].isin(['C1 ', 'C2 '])][listVars].describe()

In [ ]:
# average means for <75% percentile 
responses[(responses['response']<1224) & (responses['dohcategory'].isin(['C1 ', 'C2 ']))][listVars].describe()

In [ ]:
responses[(responses['response']>=1224) & (responses['dohcategory'].isin(['C1 ', 'C2 ']))][listVars].describe()

In [ ]:
df['callstart'].dt.hour.plot

In [ ]:
df['callHour'] = df['callstart'].dt.hour
df.head()

In [ ]:
responsesGroupedbyHour = df.groupby('callHour')
avgResponseTimebyHour = df.groupby('callHour').mean()['response']

In [ ]:
responseCountbyHour = responsesGroupedbyHour.count()['response']

In [ ]:
(avgResponseTimebyHour).plot()
plt.pyplot.ylabel('Average Response Time (sec)')
plt.pyplot.xlabel('Hour of Day')
plt.pyplot.title('Average Response Time by Hour of Day')

In [ ]:
(avgResponseTimebyHour*responseCountbyHour/sum(responseCountbyHour)).plot()
plt.pyplot.ylabel('Weighted Average Response Time')
plt.pyplot.xlabel('Hour of Day')
plt.pyplot.title('Average Response Time by Hour of Day')

In [ ]:
x = responses[(responses['wardID']=="00BJGJ")].sort_values(['lsoa'])

In [ ]:
x[['lsoa','lat_activation','lon_activation','lat_incident','lon_incident']].drop_duplicates()